# This notebook consists of generating both static and temporal segment tolerances using only one/more segment-level zernike aberrations

### Importing necessary python libraries, and PASTIS pre-built functions

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
from shutil import copy
from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np
import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 
from pastis.e2e_simulators.generic_segmented_telescopes import SegmentedAPLC
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet
from astropy.io import fits as pf
from pastis.analytical_pastis.temporal_analysis import req_closedloop_calc_batch
from matplotlib.colors import TwoSlopeNorm
import matplotlib.ticker as ticker

### Set some initial parameter or call them from config file 

In [ ]:
coronagraph_design = 'small'
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

### Define and create directory 

In [ ]:
data_dir = "/Users/asahoo/Documents/ultra/segment_zernike"
repo_dir = "/Users/asahoo/repos/PASTIS"
overall_dir = util.create_data_path(data_dir, telescope='luvoir_'+coronagraph_design)
resDir = os.path.join(overall_dir, 'matrix_numerical')

os.makedirs(resDir, exist_ok=True)

### Instantiate LUVOIR-A 

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling)

### Create segement level defocus mirror 

In [ ]:
luvoir.create_segmented_mirror(5) 

In [ ]:
luvoir.sm
n_MID = luvoir.sm.num_actuators

### Flatten the DM 

In [ ]:
luvoir.sm.flatten()

### Calculate the unaberrated coronagraphic PSF

In [ ]:
unaberrated_coro_psf, ref= luvoir.calc_psf(ref=True, display_intermediate=True, norm_one_photon=True)

### Calculate peak value of reference psf and static coronagraphic floor 

In [ ]:
norm = np.max(ref)
dh_intensity = (unaberrated_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print(f'norm: {norm}',f'constrast floor: {contrast_floor}')

### Poking each segment with a zernike defocus

In [ ]:
nonaberrated_coro_psf, ref, efield = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
print('Generating the E-fields for harris modes in science plane')
print(f'Calibration aberration used: {nm_aber} m')

start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for i in range(0, n_MID):
    print(f'Working on "bulk" thermal mode, segment: {i}')
    
    # Apply calibration aberration to used mode
    sm_mode = np.zeros(n_MID)
    #sm_mode[6*i -3] = (nm_aber)/2 
    sm_mode[i] = (nm_aber)/2
    luvoir.sm.actuators  = sm_mode
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)
    
    

In [ ]:
mat_zernike = np.zeros([n_MID, n_MID])
for i in range(0, n_MID):
    for j in range(0, n_MID):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        #print(contrast)
        mat_zernike[i, j] = contrast

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
clist = [(0.1, 0.6, 1.0), (0.05, 0.05, 0.05), (0.8, 0.5, 0.1)]
blue_orange_divergent = LinearSegmentedColormap.from_list("custom_blue_orange", clist)

plt.figure(figsize=(10,8))
norm_mat = TwoSlopeNorm(vcenter=0, vmin=-1*1e-10, vmax=1*1e-10)                                                         
plt.imshow((mat_zernike),cmap=blue_orange_divergent, norm = norm_mat)
plt.title(r"PASTIS matrix $M$ for defocus zernike", fontsize=20)
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("Mode Index",fontsize=20)
plt.tick_params(labelsize=15)
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label(r"in units of $1/{nm^2}$",fontsize =15)
plt.tight_layout()

In [ ]:
filename_matrix1 = 'PASTISmatrix_n_harris_' + str(n_MID)
hcipy.write_fits(mat_zernike, os.path.join(resDir, filename_matrix1 + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix1 + '.fits','\n'))

filename_matrix2 = 'EFIELD_Re_matrix_n_harris_' + str(n_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix2 + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix2 + '.fits', '\n'))

filename_matrix3 = 'EFIELD_Im_matrix_n_harris_' + str(n_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix3 + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix3 + '.fits','\n'))

In [ ]:
evals, evecs = np.linalg.eig(mat_zernike)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

In [ ]:
c_target_log = -11
c_target = 10**(c_target_log)
n_repeat = 20

In [ ]:
mu_map_zernike = np.sqrt(((c_target) / (n_MID)) / (np.diag(mat_zernike)))
np.savetxt('/Users/asahoo/Documents/ultra/segment_zernike/mu_map_zernike_1e-11.csv', mu_map_zernike, delimiter=',')

In [ ]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 
N_pup_z = int(luvoir.pupil_grid.shape[0] / z_pup_downsample) #N_pup_z = 100,used to define out-of-band efield
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

npup = int(np.sqrt(luvoir.pupil_grid.x.shape[0]))
nimg = int(np.sqrt(luvoir.focal_det.x.shape[0]))

# Getting the flux together
sptype = 'A0V'
Vmag = 5.0
minlam = 500
maxlam = 600 
dark_current = 0     
CIC = 0            
star_flux = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=Vmag, minlam=minlam, maxlam=maxlam) #ph/s/m^2
Nph = star_flux.value*15**2*np.sum(luvoir.apodizer**2) / npup**2

In [ ]:
luvoir.sm.flatten()
nonaberrated_coro_psf ,refshit ,inter_ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
luvoir.sm.flatten()
defocus_ref2 = luvoir.calc_out_of_band_wfs(norm_one_photon=True) #returns wavefront on obwfs detector
defocus_ref2_sub_real = hcipy.field.subsample_field(defocus_ref2.real, z_pup_downsample, grid_zernike, statistic='mean')
defocus_ref2_sub_imag = hcipy.field.subsample_field(defocus_ref2.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref_OBWFS = (defocus_ref2_sub_real + 1j*defocus_ref2_sub_imag) * z_pup_downsample

In [ ]:
nyquist_sampling = 2.

# Actual grid for LUVOIR images
grid_test = hcipy.make_focal_grid(
            luvoir.sampling,
            luvoir.imlamD,
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )

# Actual grid for LUVOIR images that are nyquist sampled
grid_det_subsample = hcipy.make_focal_grid(
            nyquist_sampling,
            np.floor(luvoir.imlamD),
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )
n_nyquist = int(np.sqrt(grid_det_subsample.x.shape[0]))

In [ ]:
design = 'small'

dh_outer_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['owa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_inner_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['iwa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_mask_nyquist = (dh_outer_nyquist - dh_inner_nyquist).astype('bool')

dh_size = len(np.where(luvoir.dh_mask != 0)[0])
dh_size_nyquist = len(np.where(dh_mask_nyquist != 0)[0])
dh_index = np.where(luvoir.dh_mask != 0)[0]
dh_index_nyquist = np.where(dh_mask_nyquist != 0)[0]

### Defining reference electric field at obwfs plane

In [ ]:
E0_OBWFS = np.zeros([N_pup_z*N_pup_z,1,2])
E0_OBWFS[:,0,0] = Efield_ref_OBWFS.real
E0_OBWFS[:,0,1] = Efield_ref_OBWFS.imag

### Defining coronagraphic reference electric field at the focal plane

In [ ]:
E0_coron = np.zeros([nimg*nimg,1,2])
E0_coron[:,0,0] = Efield_ref.real 
E0_coron[:,0,1] = Efield_ref.imag

In [ ]:
filename_matrix2 = 'EFIELD_Re_matrix_n_harris_' + str(n_MID) + '.fits'
G_zernike_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix2)) 
filename_matrix3 = 'EFIELD_Im_matrix_n_harris_' + str(n_MID) + '.fits'
G_zernike_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix3)) 

### Defining Sensitvity matrix after coronagraph plane/ image plane??

In [ ]:
G_coron_zernike= np.zeros([nimg*nimg,2,n_MID])
for pp in range(0, n_MID):
    G_coron_zernike[:,0,pp] = G_zernike_real[pp] - Efield_ref.real
    G_coron_zernike[:,1,pp] = G_zernike_imag[pp] - Efield_ref.imag

### Calculating out of band sensitvity matrix or G_OBWFS

In [ ]:
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for i in range(1, n_MID):
    #print(f'Working on "defocus" zernike mode, segment: {i}')
    
    # Apply calibration aberration to used mode
    sm_mode = np.zeros(n_MID)
    #sm_mode[6*i-3] = (nm_aber)/2 
    sm_mode[i] = (nm_aber)/2
    luvoir.sm.actuators  = sm_mode
    zernike_meas = luvoir.calc_out_of_band_wfs(norm_one_photon=True)
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_harris_' + str(n_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_harris_' + str(n_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_harris_' + str(n_MID)+'.fits'
G_OBWFS_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_harris_' + str(n_MID)+'.fits'
G_OBWFS_imag =  fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

In [ ]:
G_OBWFS= np.zeros([N_pup_z*N_pup_z,2,n_MID])
for pp in range(0, n_MID-1):
    G_OBWFS[:,0,pp] = G_OBWFS_real[pp]*z_pup_downsample - Efield_ref_OBWFS.real
    G_OBWFS[:,1,pp] = G_OBWFS_imag[pp]*z_pup_downsample - Efield_ref_OBWFS.imag

In [ ]:
flux = Nph
Qzernike = np.diag(np.asarray(mu_map_zernike**2))

Ntimes = 20
TimeMinus = -2
TimePlus = 5.5 #3.5
Nwavescale = 8
Nflux = 3

res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
result_wf_test =[]

#i=-1
for wavescale in range (1,15,2):
    #i=i+1
    print('Harris modes with batch OBWFS and noise %f'% wavescale, "i",i)  
    niter = 10
    timer1 = time.time()
    StarMag = 0.0
    #j=-1
    for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
        j=j+1
        Starfactor = 10**(-StarMag/2.5)
        print(tscale)
        tmp0 = req_closedloop_calc_batch(G_coron_zernike, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
                                                 dark_current+CIC/tscale, tscale, flux*Starfactor, 0.0001*wavescale**2*Qzernike,
                                                 niter, luvoir.dh_mask, norm)    
        tmp1 = tmp0['averaged_hist']
        n_tmp1 = len(tmp1)
        result_wf_test.append(tmp1[n_tmp1-1])

In [ ]:
delta_wf = []
for wavescale in range (1,15,2):
    wf = 1e3*np.sqrt(0.0001*wavescale**2)
    delta_wf.append(wf)

texp = np.logspace(TimeMinus, TimePlus, Ntimes)

font = {'family': 'serif','color' : 'black','weight': 'normal','size'  :  20}
plt.figure(figsize =(15,10))

plt.title('Target contrast = %s, Vmag= %s'%(c_target, Vmag),fontdict=font)
plt.plot(texp,result_wf_test[0:20]-contrast_floor, label=r'$\Delta_{wf}= %d\ pm$'%(delta_wf[0]))
plt.plot(texp,result_wf_test[20:40]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[1]))
plt.plot(texp,result_wf_test[40:60]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[2]))
plt.plot(texp,result_wf_test[60:80]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[3]))
plt.plot(texp,result_wf_test[80:100]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[4]))
plt.plot(texp,result_wf_test[100:120]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[5]))
plt.plot(texp,result_wf_test[120:140]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[6]))
plt.xlabel("$t_{WFS}$ in secs",fontsize=20)
plt.ylabel("$\Delta$ contrast",fontsize=20)
plt.yscale('log')
plt.xscale('log')
plt.legend(loc = 'upper center',fontsize=20)
plt.tick_params(top=False, bottom=True, left=True, 
                right=True,labelleft=True, labelbottom=True,
                labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',which='minor',length=6, width=2)
plt.grid()
plt.savefig('/Users/asahoo/Documents/ultra/segment_zernike/zernike_cont_wf.png')
plt.show()

### Codeblock to plot the tolerance maps in pm

In [ ]:
N_zernike = 5 
zernike_coeffs_numaps = np.zeros([N_zernike, n_MID])

for qq in range(N_zernike):
    zernike_coeffs_tmp = np.zeros([n_MID])
    for kk in range(nb_seg):
        zernike_coeffs_tmp[qq+(kk)*N_zernike] = mu_map_zernike[qq+(kk)*N_zernike]
    zernike_coeffs_numaps[qq] = zernike_coeffs_tmp #arranged 600 elements into 5*120 elements 
    
zernike_coeffs_table = np.zeros([N_zernike, nb_seg])
for qq in range(N_zernike):
    for kk in range(nb_seg):
        zernike_coeffs_table[qq,kk] = mu_map_zernike[qq+(kk)*N_zernike] #numpy ndarray 120

nu_maps = []
for qq in range(N_zernike):
    zernike_coeffs = zernike_coeffs_numaps[qq]
    luvoir.sm.actuators = zernike_coeffs*nm_aber/ 2 # m
    nu_maps.append(luvoir.sm.surface)

In [ ]:
plt.figure(figsize=(40,20))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("Segment Level Piston", fontsize=30)
plot_norm1 = TwoSlopeNorm(vcenter=4, vmin = 0, vmax=8) 
hcipy.imshow_field((nu_maps[0])*1e12, cmap = 'RdBu', norm = plot_norm1)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Segment Level X-Tilt ",fontsize=30)
plot_norm2 = TwoSlopeNorm(vcenter=0, vmin = -5, vmax=5)
hcipy.imshow_field((nu_maps[1])*1e12,cmap = 'RdBu',norm = plot_norm2)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Segment Level Y-Tilt ",fontsize=30)
plot_norm3 = TwoSlopeNorm(vcenter=0, vmin = -5, vmax=5) 
hcipy.imshow_field((nu_maps[2])*1e12,cmap = 'RdBu',norm = plot_norm3)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Segment Level Defocus",fontsize=30)
plot_norm4 = TwoSlopeNorm(vcenter=0, vmin = -10, vmax=10) 
hcipy.imshow_field((nu_maps[3])*1e12,cmap = 'RdBu', norm = plot_norm4)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Segment Level Astigmatism ",fontsize=30)
plot_norm5 = TwoSlopeNorm(vcenter=0, vmin = -10, vmax=10)
hcipy.imshow_field((nu_maps[4])*1e12,cmap = 'RdBu', norm = plot_norm5)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

plt.tight_layout()
plt.savefig('/Users/asahoo/Documents/ultra/segment_zernike/zernike_mu_map_pm.png')

### random uncorrelated noise

In [ ]:
luvoir_test = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
luvoir_test.create_segmented_mirror(5) 
luvoir_test.sm

In [ ]:
del_contrast = []
aberrated_contrast = []
rms_wfs = []
for i in range(1, 1000):    
    sm_mode = np.zeros(n_MID)
    sm_mode = np.random.normal(0, zernike_coeffs_numaps[4]*nm_aber, n_MID)
    rms = np.sqrt(np.mean(sm_mode**2))*1e12*np.sqrt(5)
    luvoir_test.sm.actuators  = sm_mode/2
    aberrated_coro_psf_t, inter_t = luvoir_test.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focal_int = np.zeros((115,115))
    focal_int = np.array(np.reshape(aberrated_coro_psf_t.amplitude,(115,115)))
    focal_cont = (np.square(focal_int))/norm
    dh_intensity = ((np.square(aberrated_coro_psf_t.amplitude))/ norm) * luvoir_test.dh_mask
    contrast_floor_aber = np.mean(dh_intensity[np.where(luvoir_test.dh_mask != 0)])
    delta_contrast = contrast_floor_aber-contrast_floor

    del_contrast.append(delta_contrast)
    aberrated_contrast.append(contrast_floor_aber)
    rms_wfs.append(rms)
    
    print(rms, contrast_floor_aber, delta_contrast)

    textstr1 = '\n'.join(("RMS (in pm) = %.2f" % (np.sqrt(np.mean(sm_mode**2))*1e12),))
    textstr2 = r'${\langle c\rangle}_{DH} = $'+ str("{:.2e}".format(contrast_floor_aber))
    textstr3 = r'$\Delta_{{\langle c\rangle}_{DH}} = $'+ str("{:.2e}".format(delta_contrast))

    plt.figure(figsize=(24,10))            
    ax1=plt.subplot(1,2,1)
    plt.title("Wavefront", fontsize =20)
    props = dict(boxstyle='round', facecolor='white', alpha=1.0)
    props2 = dict(boxstyle='round', facecolor='red', alpha=1.0)
    ax1.text(0.01, 0.99, textstr1, transform=ax1.transAxes, fontsize=15,verticalalignment='top', bbox=props)
    ax1.text(1.15, 1.01, "pm", transform=ax1.transAxes, fontsize=15,verticalalignment='top')
    plt_norm = TwoSlopeNorm(vcenter=0, vmin = -0.2, vmax=0.2)
    hcipy.imshow_field((inter_t['seg_mirror']).phase*1000, mask=luvoir_test.aperture, cmap='RdBu', norm=plt_norm)
    plt.tick_params(top=False, bottom=True, left=True, right=False ,labelleft=True, labelbottom=True, labelsize=15)
    plt.tick_params(axis='both',which='major',length=10, width=2)
    plt.tick_params(axis='both',which='minor',length=6, width=2)
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label("pm",fontsize =20)


    ax2 = plt.subplot(1,2,2)
    plt.title("Coronagraphic PSF", fontsize =20)
    props = dict(boxstyle='round', facecolor='white', alpha=1.0)
    ax2.text(0.01, 0.99, textstr2, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    ax2.text(0.01, 0.93, textstr3, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    plt.imshow(np.log10(focal_cont),cmap='magma')
    cbar = plt.colorbar(ticks = np.linspace(-14,-2,6,endpoint=False))
    cbar.ax.set_yticklabels([r'$10^{-14}$', r'$10^{-12}$', r'$10^{-10}$', 
                             r'$10^{-8}$', r'$10^{-6}$',r'$10^{-4}$'], fontsize=15)
    plt.tick_params(top=False, bottom=True, left=True, right=False ,labelleft=True, labelbottom=True, labelsize=15)
    plt.tick_params(axis='both',which='major',length=10, width=2)
    plt.tick_params(axis='both',which='minor',length=6, width=2)

    #plt.show()
    plt.savefig('/Users/asahoo/Documents/ultra/segment_zernike/astigmatism/plot_%d.png'%i)
    plt.close()
    #pf.writeto('/Users/asahoo/Desktop/data_repos/harris_data/ball_del_06/pupil_%d.fits'%pp,pupil_phase)
    


In [ ]:
import scipy

mean = np.sqrt(np.mean((zernike_coeffs_numaps[4]*nm_aber)**2))*1e12*np.sqrt(5)
#mean = np.sqrt(np.mean((mu_map_harris*nm_aber)**2))*1e12
print(mean)

plt.figure(figsize =(45,15))
plt.subplot(1,3,1)
_, bins, _ = plt.hist(rms_wfs, 100, density=1, alpha=1)
mu, sigma = scipy.stats.norm.fit(rms_wfs)
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
plt.plot(bins, best_fit_line, lw='4',label = r'$\mu$ =%.2f, $\sigma$= %.2f'%(mu,sigma))
plt.axvline(mean, c='r', ls='-.', lw='3', label='rms = %.2f' %mean)
plt.xlabel("RMS Wavefront Error (in pm)", fontsize =20)
plt.ylabel("Frequency",fontsize =15)
plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
plt.legend(fontsize=25)
#plt.xlim(0,3)

plt.subplot(1,3,2)
_, bins, _ = plt.hist(np.log10(aberrated_contrast), 100, density=1, alpha=1)
mu, sigma = scipy.stats.norm.fit(np.log10(aberrated_contrast))
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
plt.plot(bins, best_fit_line, lw='4',label = r'$\mu$ =%.2f, $\sigma$= %.2f'%(mu,sigma))
#plt.axvline(np.log10(contrast_floor), c='r', ls='-.', lw='3')
plt.xlabel("Mean contrast in DH",fontsize =15)
plt.ylabel("Frequency",fontsize =15)
plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
plt.legend(fontsize=25)
plt.xlim(-10.37,-10.33)

plt.subplot(1,3,3)
_, bins, _ = plt.hist(np.log10(del_contrast), 100, density=1, alpha=1)
mu, sigma = scipy.stats.norm.fit(np.log10(del_contrast))
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
plt.plot(bins, best_fit_line, lw='4',label = r'$\mu$ =%.2f, $\sigma$= %.2f'%(mu,sigma))
#plt.axvline(np.log10(c_target), c='r', ls='-.', lw='3')
plt.xlabel("Change in contrast in DH",fontsize =20)
plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
plt.ylabel("Frequency",fontsize =15)
plt.legend(fontsize=25)
plt.savefig('/Users/asahoo/Documents/ultra/segment_zernike/hist_astigmatism.png')

In [ ]:
from PIL import Image
import glob

imgs = glob.glob("/Users/asahoo/Documents/ultra/segment_zernike/astigmatism/*.png")
imgs.sort()
frames = []

for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
    
#Save into a GIF file that loops forever
frames[0].save('/Users/asahoo/Documents/ultra/segment_zernike/mu_map_astigmatism.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=500, loop=0)

In [ ]:
# for i in range(1, 121):
#     print(f'Working on "bulk" thermal mode, segment: {i+1}')
    
#     # Apply calibration aberration to used mode
#     sm_mode = np.zeros(n_MID)
#     sm_mode[6*i -3] = (nm_aber)/2 
#     luvoir.sm.actuators  = sm_mode
    
#     plt.figure()
#     plt.text(9.6, 7.6, 'm', fontsize = 10)
#     hcipy.imshow_field(luvoir.sm.surface, mask=luvoir.aperture, cmap='RdBu')
#     cbar = plt.colorbar()
#     plt.tight_layout()
#     #plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/poke_%d.png'% i, dpi=165)
#     plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/defocus_plots/poke_{0:03}.png'.format(i), dpi=165)